# Парсинг данных

В рамках данного проекта необходимо осуществить парсинг данных с веб-сайта. В качества веб-сайта взят habr.com. Необходимо осуществить парсинг лучших постов за все время. Данные для парсинга: 1) параметры - наименование статьи, дата загрузки, категория, к которой пост относится, и 2) целевой показатель - число просмотров. 

В дальнейшем можно рассмотреть не только лучшие за все ремя посты, но и увеличить выборку еще большим объемом повседневных постов.

На данный момент объем выборки должен составить 1000 статей (20 объектов на странице, всего 50 страниц).

<br>**План проекта**:
1. Импортирование библиотек
2. Парсинг данных
3. Сохранение базы данных

## 1. Импортирование библиотек

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd

'''
from bs4 import BeautifulSoup
import os
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from itertools import cycle
'''

'\nfrom bs4 import BeautifulSoup\nimport os\nimport concurrent.futures\nfrom concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor\nfrom itertools import cycle\n'

## 2. Парсинг данных

In [2]:
# Ссылка на Habr и открытие браузера Chrome
habr_url = 'https://habr.com/ru/top/alltime/'
driver = webdriver.Chrome()

In [3]:
# Функция для получения данных с одной веб-страницы
def get_page_data(page_num):
    # Заход на веб-страницу
    page_url = f'{habr_url}page{page_num}'
    driver.get(page_url)
    time.sleep(2)
    
    # Прокрутка до конца страницы
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Наименования постов
    titles = [title.text for title in driver.find_elements(By.CLASS_NAME, 'tm-title__link')]
    
    # Количество просмотров
    view_counts_link = driver.find_elements(By.CLASS_NAME, 'tm-icon-counter__value')
    view_counts = []
    for view_count in view_counts_link:
        if 'M' in view_count.text:
            view_counts.append(float(re.split('M', view_count.text)[0]) * 1000)
        elif 'K' in view_count.text:
            view_counts.append(float(re.split('K', view_count.text)[0]))
    
    # Категории
    categories_link = driver.find_elements(By.CLASS_NAME, 'tm-article-snippet__hubs-container')
    categories = []
    for category in categories_link:
        categories.append([i for i in re.split('\n', category.text) if i != '*'])

    # Дата и время публикации
    dates_link = driver.find_elements(By.CSS_SELECTOR, 'time')
    dates = []
    for date in driver.find_elements(By.CSS_SELECTOR, 'time'):
        if 'в' in date.text:
            dates.append(date.get_attribute('datetime'))
    
    df = pd.concat(
        [pd.Series(titles[:20]), pd.Series(view_counts[:20]), pd.Series(categories[:20]), pd.Series(dates[:20])],
        axis=1
    )
    
    df.columns = ['title', 'view_count', 'category', 'date']
    
    return df

In [4]:
df_posts = pd.DataFrame(columns=['title', 'view_count', 'category', 'date'])

for n in range(1, 51):
    df_posts = df_posts.append(get_page_data(n), ignore_index=True)

In [7]:
df_posts.tail()

,title,view_count,category,date
995,Ликбез по типизации в языках программирования,459.0,[Программирование],2012-12-03T16:46:41.000Z
996,Компьютер-флэшка за $25,28.0,[DIY или Сделай сам],2011-05-06T09:01:18.000Z
997,Очередная регистрация ООО или жестокая война с...,15.0,[Законодательство в IT],2011-03-08T21:09:34.000Z
998,По сравнению с тобой большинство людей кажутся...,12.0,[Интерфейсы],2010-10-26T12:38:41.000Z
999,Как же стать злым?,7.7,[IT-компании],2010-09-15T12:13:08.000Z


## 3. Сохранение данных

In [6]:
df_posts.to_csv('posts_data.csv')